In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
    # for filename in filenames:
        # pass
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Data

In [ ]:
import os
from glob import glob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import distance, distance_matrix 

In [ ]:
def imgLoader(path):
    imgBL = Image.open(path).convert("L")
    arr = np.array(imgBL)
    arr = np.where(arr<255,0,1)
    return arr.astype("uint8")

# plt.subplots(figsize=(25,12)) # for banner
plt.imshow(
    np.concatenate(
        [np.array(imgLoader("../input/candle-stick-patterns/cs3_0.png")),
        np.array(imgLoader("../input/candle-stick-patterns/cs3_10.png")),
        np.array(imgLoader("../input/candle-stick-patterns/cs3_20.png")),
        np.array(imgLoader("../input/candle-stick-patterns/cs3_30.png")),
        np.array(imgLoader("../input/candle-stick-patterns/cs3_40.png"))],
        1
    ), "gray")
plt.axis("off")

In [ ]:
%%time

file_path = glob("../input/candle-stick-patterns/*.png")
data_df = pd.DataFrame(file_path, columns=["file_path"])
data_df["imgID"] = data_df["file_path"].str.extract("_(\d+)\.png").astype(int)
data_df["imgData"] = data_df["file_path"].apply(imgLoader)


In [ ]:
data_df.to_pickle("data_df.pkl")
print(data_df.shape)
data_df[:3]

# EDA

In [ ]:
mat = data_df.sort_values("imgID")["imgData"].apply(lambda x: x.reshape(1,-1))
mat = np.concatenate(mat.values)
mat[:3]

## Identify Cluster Distribution

In [ ]:
# sns.heatmap(mat[:1600,:1600])

# sns.clustermap(mat[:1600,:1600], col_cluster=False)

In [ ]:
%%time

dist_mat = distance_matrix(mat[:1600,:1600], mat[:1600,:1600], p=1)
print(dist_mat.shape)
dist_mat[:5,:5]

In [ ]:
sns.clustermap(dist_mat)

# Try AutoEncoder

In [ ]:
import keras
from keras import layers

In [ ]:
def define_model():
    keras.backend.clear_session()

    input_img = keras.Input(shape=(40, 40, 1))

    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = keras.Model(input_img, decoded)
    return encoded, decoded, autoencoder

encoded, decoded, autoencoder = define_model()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
X = data_df.sort_values("imgID")["imgData"].apply(lambda x: x.reshape(1,40,40,1))
X = np.concatenate(X.values)
X.shape

In [ ]:
autoencoder.fit(X, X, 
                batch_size=32,
                epochs=3,
                verbose=1,
                validation_split=0.1)

ref. https://blog.keras.io/building-autoencoders-in-keras.html